<a href="https://colab.research.google.com/github/ailab-nda/NLP/blob/main/Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日本語学習済みモデルによる自然言語処理

### 準備（関連ライブラリのインストール）

In [7]:
!pip install -q transformers
!pip install -q sentencepiece

     |████████████████████████████████| 2.9 MB 15.8 MB/s 
     |████████████████████████████████| 895 kB 79.8 MB/s 
     |████████████████████████████████| 3.3 MB 67.3 MB/s 
     |████████████████████████████████| 596 kB 70.0 MB/s 
     |████████████████████████████████| 56 kB 3.7 MB/s 
     |████████████████████████████████| 1.2 MB 13.4 MB/s 


## 1. RoBERTa による文章中の空欄埋め



### モデルのダウンロード

In [57]:
from transformers import T5Tokenizer, RobertaForMaskedLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-roberta-base")
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

model = RobertaForMaskedLM.from_pretrained("rinna/japanese-roberta-base")

### 問題文の作成

In [60]:
# original text
#text = "4年に1度オリンピックは開かれる。"
text = ""

# prepend [CLS]
text = "[CLS]" + text

# tokenize
tokens = tokenizer.tokenize(text)
print(tokens)

# mask a token
masked_idx = 5
tokens[masked_idx] = tokenizer.mask_token
print(tokens)  # output: ['[CLS]', '▁4', '年に', '1', '度', '[MASK]', 'は', '開かれる', '。']

['[CLS]', '▁4', '年に', '1', '度', 'オリンピック', 'は', '開かれる', '。']
['[CLS]', '▁4', '年に', '1', '度', '[MASK]', 'は', '開かれる', '。']


### 穴埋め問題を解く

補充すべき単語の推定 (id)

In [61]:
# convert to ids
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

# convert to tensor
import torch
token_tensor = torch.LongTensor([token_ids])

[4, 1602, 44, 24, 368, 6, 11, 21583, 8]


結果の表示

In [63]:
# provide position ids explicitly
position_ids = list(range(0, token_tensor.size(1)))
position_id_tensor = torch.LongTensor([position_ids])

# get the top 10 predictions of the masked token
with torch.no_grad():
    outputs = model(input_ids=token_tensor, position_ids=position_id_tensor)
    predictions = outputs[0][0, masked_idx].topk(10)

for i, index_t in enumerate(predictions.indices):
    index = index_t.item()
    token = tokenizer.convert_ids_to_tokens([index])[0]
    print(i, token)

0 総会
1 サミット
2 ワールドカップ
3 フェスティバル
4 大会
5 オリンピック
6 全国大会
7 党大会
8 イベント
9 世界選手権


## 2. GPT-2 による文書生成

### (1) rinna/japanese-gpt2 の利用

### モデルのダウンロード

* 大きなモデル



In [33]:
from transformers import T5Tokenizer, AutoModelForCausalLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")

* 小さなモデル

In [8]:
from transformers import T5Tokenizer, GPT2LMHeadModel

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-small")
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

model = GPT2LMHeadModel.from_pretrained("rinna/japanese-gpt2-small")

Downloading:   0%|          | 0.00/787k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/153 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/282 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/846 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433M [00:00<?, ?B/s]

### 文書生成の例

In [9]:
input = tokenizer.encode("私は防衛大学校に入校してからというもの、", return_tensors="pt")
output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=3)
sentences = tokenizer.batch_decode(output)
for i in sentences:
    print(i)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


私は防衛大学校に入校してからというもの、</s> 今までに学んできたことをきちんと伝え、そして自分が持っている能力、強みを引き出してくれるように導いてくださる先生方の姿勢にとても感謝しています。 今日学んだことで改めて自分の苦手な分野というものについて考える機会となりました。 今日教えてくださった情報交換会に参加しました。 前回に引き続き、防衛大学校 先輩 より 防衛大学校 1年生の方々の授業について頂きました。 4年生
私は防衛大学校に入校してからというもの、</s> 社会人の私に付き合ってくれることなんてなかったし、何も言うこともできなかった・・・ それよりも、自分の心に深い傷を残してしまったことを後悔してるという現実感。 もちろん、自分にとってはそれが大きな責任であったことは確かなのです。 しかし、「これがあったからこそ、俺は本気で戦えるんだ!」という思いに 社会人としての第一歩を踏み出し、社会人としての常識を少し超えた結果が 「自分は何
私は防衛大学校に入校してからというもの、</s> いまや防衛学校は軍事研究に欠かせない学問とされています。 そこで私は防衛大学校では本当にその意味が本当に必要なものだと思うようになりました。 いま思えば防衛学校の教官もそれを感じ取っていたのでしょうね。 いまはそういった情報がほとんど氾濫していますが、それでもやはり防衛大学校の教練は教えられるものだと思います。 私が入学して防衛大学校を卒業して10年目となるのですが、そこでは先輩から多くの事を


## (2) GPT2-Japanese の利用

### モデルのダウンロードとインストール

In [1]:
# gpt2-japaneseのインストール
!git clone https://github.com/tanreinama/gpt2-japanese
%cd gpt2-japanese
#!pip uninstall tensorflow -y
!pip install -r requirements.txt

Cloning into 'gpt2-japanese'...
remote: Enumerating objects: 263, done.
remote: Counting objects: 100% (222/222), done.
remote: Compressing objects: 100% (154/154), done.
remote: Total 263 (delta 121), reused 153 (delta 64), pack-reused 41
Receiving objects: 100% (263/263), 1.22 MiB | 13.86 MiB/s, done.
Resolving deltas: 100% (140/140), done.
/content/gpt2-japanese
  Created wheel for jaconv: filename=jaconv-0.3-py3-none-any.whl size=15565 sha256=7c9ac78cd3e33e06fdee0efa9b99bd1e13ec1a5a304f7b6c2009fa4371f4b0b2
  Stored in directory: /root/.cache/pip/wheels/8f/4f/c2/a2a3b14d0e94f855f4aa8887bf0267bee9ecfb8e62a9ee2d92
Successfully built jaconv


In [3]:
# smallモデルのダウンロード
!wget https://www.nama.ne.jp/models/gpt2ja-small.tar.bz2
!tar xvfj gpt2ja-small.tar.bz2

--2021-10-14 11:17:24--  https://www.nama.ne.jp/models/gpt2ja-small.tar.bz2
Resolving www.nama.ne.jp (www.nama.ne.jp)... 112.78.112.176
Connecting to www.nama.ne.jp (www.nama.ne.jp)|112.78.112.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 386498535 (369M) [application/x-bzip2]
Saving to: ‘gpt2ja-small.tar.bz2’

gpt2ja-small.tar.bz 100%[===================>] 368.59M  2.37MB/s    in 2m 35s  

2021-10-14 11:20:00 (2.39 MB/s) - ‘gpt2ja-small.tar.bz2’ saved [386498535/386498535]

gpt2ja-small/
gpt2ja-small/checkpoint
gpt2ja-small/model-10412700.data-00000-of-00001
gpt2ja-small/model-10412700.index
gpt2ja-small/model-10412700.meta
special_thanks.txt


### ランダムな文章の作成

In [13]:
# smallモデルの動作確認
!python gpt2-generate.py --model gpt2ja-small --num_generate 1

2021-10-14 11:36:32.778404: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-14 11:36:32.778475: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (309b29329430): /proc/driver/nvidia/version does not exist
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.random.categorical` instead.
「……………………」
俺は沈黙を破り、目の前の老人を見る。
「そうよ！この人が今日紹介したのが私！そう、これからもよろしくお願いしますだ！だから皆さんにはこの家の中で生きてもらう為に、この家で生活してもらうわ！」
「…………………」
老人が固まってしまった。
そして、どういう事なのかと言おうと思ったときに爺が部屋中に入ってきたので、すぐに気づいた。
「…………………」
爺が俺を見る。
何故老人がこうなったのか、俺にはよく分からなかったが言っておきたい事があった。
「おい、何か言ってみろよ……」
俺は爺に話すべきか迷った。
爺は、俺が話すべきじゃないと思っていると、すぐにじいさんと爺さんに耳打ちをした。
「すまないが、私は今日ここに来た訳ではないんだ……。何かお勧めのモノか？…………その時に、あのお爺（じい）さんの話したお話を思い出して欲しいんだ……」
「……………………」
俺は老人の話を、聞いていた。
ここに来る前、爺は、俺が見ていたモノとは別に、こう言っていた。



### 文章の続きを作成

In [14]:
!python gpt2-generate.py --model gpt2ja-small --num_generate 3 --context="佐藤　浩は、"

2021-10-14 11:37:05.936508: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-14 11:37:05.936567: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (309b29329430): /proc/driver/nvidia/version does not exist
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.random.categorical` instead.
1989年8月26日に、神奈川県立第一高校から東京大学に進学することを発表。21歳の時に、慶應義塾大学商学部に入るが、社会学部出身の鈴木啓之（元株式会社アート担当）に、同じく社会学部卒業後の1992年に第99回選抜高等学校野球大会・神奈川県大会に出場していることが知れ、大学を中退して、1992年7月に、慶應義塾大学に入学し、同大学大学院に進む。大学院修了後の1994年に、慶應義塾大学大学院において、社会学部の社会科学系学科長に就任。2005年「未来社会科学に関する調査研究の活動」で大学院修士課程を修了、社会科学系大学院に進学し、社会学研究科教授に就任した。「社会科学研究」の講義を聞いている最中、2008年9月には、慶應義塾大学の社会科学系学科長に就任している。「社会科学研究」は、早稲田大寄贈研究員である山口進二から、社会科学研究における「社会科学的に社会科学的に社会を構成する課題」について提言されたものである。
社会科学研究においては、学部・専攻としながらも、人間社会の構造や社会構造について科学的な分析について研究している。「人間社会の構造」については、大学で研究した内容を学びながら人
この日も大きな拍手に包まれて心の中で「あ